In [16]:
"""
Uses tf.contrib.data module which is in release candidate 1.2.0rc0
Based on:
    - PyTorch example from Justin Johnson:
      https://gist.github.com/jcjohnson/6e41e8512c17eae5da50aebef3378a4c
      - https://gist.github.com/omoindrot/dedc857cdc0e680dfb1be99762990c9c
Required packages: tensorflow (v1.2)
You can install the release candidate 1.2.0rc0 here:
https://www.tensorflow.org/versions/r1.2/install/

Download the weights trained on ImageNet for VGG:
```
wget http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
tar -xvf vgg_16_2016_08_28.tar.gz
rm vgg_16_2016_08_28.tar.gz
```
"""

%matplotlib inline

from matplotlib import pyplot as plt

import argparse
import os
import datetime
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets

from vggnet_utils import *

VGG_MEAN = [123.68, 116.78, 103.94]

print(tf.__version__)

1.2.0-rc1


In [97]:
def leaky_relu(x, alpha=0.01):
    """Compute the leaky ReLU activation function.
    
    Inputs:
    - x: TensorFlow Tensor with arbitrary shape
    - alpha: leak parameter for leaky ReLU
    
    Returns:
    TensorFlow Tensor with the same shape as x
    """
    # TODO: implement leaky ReLU
    return(tf.maximum((alpha*x), x))

In [105]:
experiments = [] # list with dict of params for each experiment

lr1s = [1e-3] # 5e-5, 
lr2s = [1e-5] # 1e-6, 1e-7, 
dropout_probs = [0.4] # 0.5, 0.3
weight_decays = [1e-4] # 5e-4, 

# create a dict of experiments
for lr1 in lr1s:
    for lr2 in lr2s:
        for dp in dropout_probs:
            for wd in weight_decays:
                experiments.append({'lr1': lr1, 'lr2': lr2, 'dp': dp, 'wd': wd})

print(experiments)


# Get the list of filenames and corresponding list of labels for training et validation
# train_filenames, train_labels = list_images(args.train_dir)
# val_filenames, val_labels = list_images(args.val_dir)

all_filenames, all_labels = list_images(args.train_dir)

train_filenames, train_labels, val_filenames, val_labels = split_samples(all_filenames, all_labels)

num_classes = 17
num_val = len(val_labels)
    
# Training log
loss_log = []
f_log = []
thresh_log = []
predicted_logits = []
actual_labels = []

for params_dict in experiments:
    print(params_dict)
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_dir', default='data/train-jpg/')
    parser.add_argument('--model_path', default='vgg_16.ckpt', type=str)
    parser.add_argument('--batch_size', default=100, type=int) #32
    parser.add_argument('--num_workers', default=50, type=int) #4
    parser.add_argument('--num_epochs1', default=15, type=int) #10
    parser.add_argument('--num_epochs2', default=10, type=int) #10
    parser.add_argument('--learning_rate1', default = params_dict['lr1'], type=float) #1e-3
    parser.add_argument('--learning_rate2', default = params_dict['lr2'], type=float)
    parser.add_argument('--dropout_keep_prob', default = params_dict['dp'], type=float)
    parser.add_argument('--weight_decay', default = params_dict['wd'], type=float)
    
    # For accessing args in an ipython notebook
    import sys; sys.argv=['']; del sys
    args = parser.parse_args()


    # --------------------------------------------------------------------------
    # In TensorFlow, you first want to define the computation graph with all the
    # necessary operations: loss, training op, accuracy...
    # Any tensor created in the `graph.as_default()` scope will be part of `graph`
    graph = tf.Graph()
    with graph.as_default():
        # Standard preprocessing for VGG on ImageNet taken from here:
        # https://github.com/tensorflow/models/blob/master/slim/preprocessing/vgg_preprocessing.py
        # Also see the VGG paper for more details: https://arxiv.org/pdf/1409.1556.pdf
    
        # Preprocessing (for both training and validation):
        # (1) Decode the image from jpg format
        # (2) Resize the image so its smaller side is 256 pixels long
        def _parse_function(filename, label):
            image_string = tf.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)          # (1)
            image = tf.cast(image_decoded, tf.float32)
    
            smallest_side = 256.0
            height, width = tf.shape(image)[0], tf.shape(image)[1]
            height = tf.to_float(height)
            width = tf.to_float(width)
    
            scale = tf.cond(tf.greater(height, width),
                            lambda: smallest_side / width,
                             lambda: smallest_side / height)
            new_height = tf.to_int32(height * scale)
            new_width = tf.to_int32(width * scale)
    
            resized_image = tf.image.resize_images(image, [new_height, new_width])  # (2)
            return resized_image, label

        # Preprocessing (for training)
        # (3) Take a random 224x224 crop to the scaled image
        # (4) Horizontally flip the image with probability 1/2
        # (5) Substract the per color mean `VGG_MEAN`
        # Note: we don't normalize the data here, as VGG was trained without normalization
        def training_preprocess(image, label):
            crop_image = tf.random_crop(image, [224, 224, 3])                       # (3)
            flip_image = tf.image.random_flip_left_right(crop_image)                # (4)
    
            means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
            centered_image = flip_image - means                                     # (5)
    
            return centered_image, label
    
        # Preprocessing (for validation)
        # (3) Take a central 224x224 crop to the scaled image
        # (4) Substract the per color mean `VGG_MEAN`
        # Note: we don't normalize the data here, as VGG was trained without normalization
        def val_preprocess(image, label):
            crop_image = tf.image.resize_image_with_crop_or_pad(image, 224, 224)    # (3)
    
            means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
            centered_image = crop_image - means                                     # (4)
    
            return centered_image, label
    
            # ----------------------------------------------------------------------
            # DATASET CREATION using tf.contrib.data.Dataset
            # https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/data
        
            # The tf.contrib.data.Dataset framework uses queues in the background to feed in
            # data to the model.
            # We initialize the dataset with a list of filenames and labels, and then apply
        # the preprocessing functions described above.
        # Behind the scenes, queues will load the filenames, preprocess them with multiple
        # threads and apply the preprocessing in parallel, and then batch the data
    
        # Training dataset
        train_filenames = tf.constant(train_filenames)
        train_labels = tf.constant(train_labels)
        train_dataset = tf.contrib.data.Dataset.from_tensor_slices((train_filenames, train_labels))
        train_dataset = train_dataset.map(_parse_function,
           num_threads=args.num_workers, output_buffer_size=args.batch_size)
        train_dataset = train_dataset.map(training_preprocess,
           num_threads=args.num_workers, output_buffer_size=args.batch_size)
        train_dataset = train_dataset.shuffle(buffer_size=10000)  # don't forget to shuffle
        batched_train_dataset = train_dataset.batch(args.batch_size)
    
        # Validation dataset
        val_filenames = tf.constant(val_filenames)
        val_labels = tf.constant(val_labels)
        val_dataset = tf.contrib.data.Dataset.from_tensor_slices((val_filenames, val_labels))
        val_dataset = val_dataset.map(_parse_function,
        num_threads=args.num_workers, output_buffer_size=args.batch_size)
        val_dataset = val_dataset.map(val_preprocess,
        num_threads=args.num_workers, output_buffer_size=args.batch_size)
        batched_val_dataset = val_dataset.batch(args.batch_size)

        # Now we define an iterator that can operator on either dataset.
        # The iterator can be reinitialized by calling:
        #     - sess.run(train_init_op) for 1 epoch on the training set
        #     - sess.run(val_init_op)   for 1 epoch on the valiation set
        # Once this is done, we don't need to feed any value for images and labels
        # as they are automatically pulled out from the iterator queues.
    
        # A reinitializable iterator is defined by its structure. We could use the
        # `output_types` and `output_shapes` properties of either `train_dataset`
        # or `validation_dataset` here, because they are compatible.
        iterator = tf.contrib.data.Iterator.from_structure(batched_train_dataset.output_types,
                                                           batched_train_dataset.output_shapes)
        images, labels = iterator.get_next()
        train_init_op = iterator.make_initializer(batched_train_dataset)
        val_init_op = iterator.make_initializer(batched_val_dataset)
    
        # Indicates whether we are in training or in test mode
        is_training = tf.placeholder(tf.bool)
    
        # ---------------------------------------------------------------------
        # Now that we have set up the data, it's time to set up the model.
        # For this example, we'll use VGG-16 pretrained on ImageNet. We will remove the
        # last fully connected layer (fc8) and replace it with our own, with an
        # output size num_classes=8
        # We will first train the last layer for a few epochs.
        # Then we will train the entire model on our dataset for a few epochs.
    
        # Get the pretrained model, specifying the num_classes argument to create a new
        # fully connected replacing the last one, called "vgg_16/fc8"
        # Each model has a different architecture, so "vgg_16/fc8" will change in another model.
        # Here, logits gives us directly the predicted scores we wanted from the images.
        # We pass a scope to initialize "vgg_16/fc8" weights with he_initializer
        vgg = tf.contrib.slim.nets.vgg
        with slim.arg_scope(vgg.vgg_arg_scope(weight_decay=args.weight_decay)):
            _, layers = vgg.vgg_16(images, num_classes=num_classes, is_training=is_training,
                                       dropout_keep_prob=args.dropout_keep_prob)
            # _ output logits
        # add some extra layers after fc6
        # Out of the arg scope
        vgg_fc6_tensor = layers['vgg_16/fc6']
        # print(vgg_fc6_tensor.shape)

        with tf.variable_scope("extra_layers"):
            extra_fc1 = tf.layers.dense(vgg_fc6_tensor, 4096, name="extra_fc1", activation = tf.nn.relu, trainable = True) #1024
            extra_bn1 = tf.layers.batch_normalization(extra_fc1, axis = -1, training=True, name = "extra_bn1")
            extra_conv1 = tf.layers.conv2d(tf.reshape(extra_bn1, shape = [-1, 64, 64, 1]), filters = 32, kernel_size = [5, 5],
                                 strides = 1, activation = leaky_relu, name = "extra_conv1")
            extra_pool1 = tf.layers.max_pooling2d(extra_conv1, pool_size = [2, 2], strides = 2, name = "extra_pool1")
            extra_conv2 = tf.layers.conv2d(extra_pool1, filters = 64, kernel_size = [5, 5],
                                 strides = 1, activation = leaky_relu, name = "extra_conv2")
            extra_pool2 = tf.layers.max_pooling2d(extra_conv2, pool_size = [2, 2], strides = 2, name = "extra_pool2")
            print(extra_pool2.shape)
            extra_pool2_flat = tf.reshape(extra_pool2, [-1, 13*13*64], name = "extra_pool2_flat")
            extra_conn1 = tf.layers.dense(extra_pool2_flat, units = 13*13*64, activation = leaky_relu, name = "extra_conn1")
            logits = tf.layers.dense(extra_conn1, num_classes, name="logits", activation = tf.sigmoid, trainable = True)

            # extra_fc2 = tf.layers.dense(extra_fc1, 1024, name="extra_fc2", trainable = True)
            # extra_fc3 = tf.layers.dense(extra_fc2, 1024, name="extra_fc3", trainable = True)
            # extra_fc4 = tf.layers.dense(extra_fc3, 256, name="extra_fc4", trainable = True)
            # logits = tf.layers.dense(extra_fc4, num_classes, name="logits", activation = tf.sigmoid, trainable = True)
    
        # Specify where the model checkpoint is (pretrained weights).
        model_path = args.model_path
        assert(os.path.isfile(model_path))
    
        # Restore only the layers up to fc6 (included)
        # Calling function `init_fn(sess)` will load all the pretrained weights.
        variables_to_restore = tf.contrib.framework.get_variables_to_restore(exclude=['vgg_16/fc8', 'vgg_16/fc7', 
                                                                                      "extra_layers/extra_fc1",
                                                                                      "extra_layers/extra_fc1", 
                                                                                      "extra_layers/extra_bn1", 
                                                                                      "extra_layers/extra_conv1", 
                                                                                      "extra_layers/extra_pool1", 
                                                                                      "extra_layers/extra_conv2", 
                                                                                      "extra_layers/extra_pool2",
                                                                                      "extra_layers/extra_pool2_flat",
                                                                                      "extra_layers/extra_conn1",
                                                                                     # "extra_layers/extra_fc2",
                                                                                     # "extra_layers/extra_fc3",
                                                                                     # "extra_layers/extra_fc4",
                                                                                     # "extra_layers/extra_fc5",
                                                                                     # "extra_layers/extra_fc6",
                                                                                      "extra_layers/logits"]) #, 'vgg_16/fc6'])
        init_fn = tf.contrib.framework.assign_from_checkpoint_fn(model_path, variables_to_restore)
    
        # Initialization operation from scratch for the new "fc6" layers
        # `get_variables` will only return the variables whose name starts with the given pattern
        # fc6_variables = tf.contrib.framework.get_variables('vgg_16/fc6')
        # fc6_init = tf.variables_initializer(fc6_variables)
        
        # Initialization operation from scratch for the new "fc7" layers
        # `get_variables` will only return the variables whose name starts with the given pattern
        fc7_variables = tf.contrib.framework.get_variables('vgg_16/fc7')
        fc7_init = tf.variables_initializer(fc7_variables)
        
        # Initialization operation from scratch for the new "fc8" layers
        # `get_variables` will only return the variables whose name starts with the given pattern
        fc8_variables = tf.contrib.framework.get_variables('vgg_16/fc8')
        fc8_init = tf.variables_initializer(fc8_variables)
        
        # Initialize extra layer variables
        extras_init = tf.variables_initializer(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'extra*'))
        
        # ---------------------------------------------------------------------
        # Using tf.losses, any loss is added to the tf.GraphKeys.LOSSES collection
        # We can then call the total loss easily
        # tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits) 
        
        tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=tf.reshape(logits, shape = [-1, num_classes])) # softmax cross entropy loss so can have labels with multiple classes
        loss = tf.losses.get_total_loss()
        
        tf.summary.scalar("loss", loss) # log the loss in summary
    
        # First we want to train only the reinitialized last layer fc8 for a few epochs.
        # We run minimize the loss only with respect to the fc8 variables (weight and bias).
        # fc876_optimizer = tf.train.GradientDescentOptimizer(args.learning_rate1)
        # fc876_train_op = fc876_optimizer.minimize(loss, var_list=[fc8_variables, fc7_variables, fc6_variables])
    
    
        # First we want to train only the last extra layers for a few epochs.
        # We minimize the loss only with respect to the extra variables (weight and bias).
        extra_optimizer = tf.train.GradientDescentOptimizer(args.learning_rate1)
        extra_train_op = extra_optimizer.minimize(loss, var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'extra*')) # [logits, extra_fc1, extra_fc2]) ### fc8_variables, fc7_variables])
        # print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'extra*'))
        
        
        # Then we want to finetune the entire model for a few epochs.
        # We run minimize the loss only with respect to all the variables.
        full_optimizer = tf.train.GradientDescentOptimizer(args.learning_rate2)
        full_train_op = full_optimizer.minimize(loss)
        
   
        # Evaluation metrics
        # Merge all logged summaries together
        merged_summary_op = tf.summary.merge_all()
    
        tf.get_default_graph().finalize()
    
    # --------------------------------------------------------------------------
    # Now that we have built the graph and finalized it, we define the session.
    # The session is the interface to *run* the computational graph.
    # We can call our training operations with `sess.run(train_op)` for instance
    with tf.Session(graph=graph) as sess:
    
        now = datetime.datetime.now()
        # start summary of session
        train_writer = tf.summary.FileWriter('./summaries/' +
                                             '{}-{}-{}-{}-{}'.format(now.year, now.month, now.day, now.hour, now.minute) , sess.graph)
        # sess.run(tf.global_variables_initializer())
        init_fn(sess)  # load the pretrained weights
        
        # For debugging
        # for v in tf.global_variables():
        #     print(v.name)
        
        sess.run(extras_init) # initialize extra variables
        sess.run(fc8_init)  # initialize the unused fc8 layer
        sess.run(fc7_init) # initialize unused fc7
        # sess.run(fc6_init) # initialize fc6
        
        iteration = 0
        # Update only the last layer for a few epochs.
        for epoch in range(args.num_epochs1):
            # Run an epoch over the training data.
            print('Starting epoch %d / %d' % (epoch + 1, args.num_epochs1))
            # Here we initialize the iterator with the training set.
            # This means that we can go through an entire epoch until the iterator becomes empty.
            sess.run(train_init_op)
            while True: # and iteration < 10:
                try:
                    _, curr_loss, curr_logits, curr_labels, summary = sess.run([extra_train_op, loss, logits, labels, merged_summary_op], {is_training: True}) # fc876_train_op,
                    loss_log.append(curr_loss)
                    train_writer.add_summary(summary, iteration)
                    curr_logits = np.reshape(curr_logits, [-1, num_classes])
                    
                    if curr_loss < 100:
                        print('Current loss: %f' % curr_loss)

                        # Check F score on logits
                        threshs = [0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95]
                        Fs = []
                        
                        for i in range(len(threshs)):
                            Fs.append(fbeta_score(curr_labels, np.array(curr_logits) > threshs[i], beta=2, average='samples'))
                        max_i = Fs.index(max(Fs))
                        print('thresh: %f, max F: %f' %(threshs[max_i], Fs[max_i]))
                        thresh_log.append(threshs[max_i])
                        f_log.append(Fs[max_i])
                        
                        iteration += 1
                    else:
                        print("EXPLODING LOSS")
                        break
                except tf.errors.OutOfRangeError:
                    break
            
            # Print current loss
            print('Current epoch loss: %f' % curr_loss)
            
            # Check F score on logits
            threshs = [0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95]
            Fs = []
            for i in range(len(threshs)):
                Fs.append(fbeta_score(curr_labels, np.array(curr_logits) > threshs[i], beta=2, average='samples'))
            max_i = Fs.index(max(Fs))
            print('Epoch thresh: %f, max F: %f' %(threshs[max_i], Fs[max_i]))
            
            # Check accuracy on the train and val sets every epoch.
            # train_acc = check_accuracy(sess, correct_prediction, is_training, train_init_op)
            # val_acc = check_accuracy(sess, correct_prediction, is_training, val_init_op)
            # print('Train accuracy: %f' % train_acc)
            # print('Val accuracy: %f\n' % val_acc)
    
    
        # Train the entire model for a few more epochs, continuing with the *same* weights.
        for epoch in range(args.num_epochs2):
            print('Starting epoch %d / %d' % (epoch + 1, args.num_epochs2))
            sess.run(train_init_op)
            while True: #  and iteration < 20:
                try:
                    _, curr_loss, curr_logits, curr_labels, summary = sess.run([full_train_op, loss, logits, labels, merged_summary_op], {is_training: True})
                    loss_log.append(curr_loss)
                    train_writer.add_summary(summary, iteration)

                    curr_logits = np.reshape(curr_logits, [-1, num_classes])

                    if curr_loss < 100:
                        print('Current loss: %f' % curr_loss)

                        # Check F score on logits
                        threshs = [0.5, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95]
                        Fs = []
                        for i in range(len(threshs)):
                            Fs.append(fbeta_score(curr_labels, np.array(curr_logits) > threshs[i], beta=2, average='samples'))
                        max_i = Fs.index(max(Fs))
                        print('thresh: %f, max F: %f' %(threshs[max_i], Fs[max_i]))
                        thresh_log.append(threshs[max_i])
                        f_log.append(Fs[max_i])
                        
                        if epoch == (args.num_epochs2-1):
                            predicted_logits.append(curr_logits)
                            actual_labels.append(curr_labels)
                        
                        iteration += 1
                    else:
                        print("EXPLODING LOSS")
                        break
                except tf.errors.OutOfRangeError:
                    break
    
            # Print current loss
            print('Entire model epoch loss: %f' % curr_loss)
    
            # Check F score on logits
            threshs = [0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95]
            Fs = []
            for i in range(len(threshs)):
                Fs.append(fbeta_score(curr_labels, np.array(curr_logits) > threshs[i], beta=2, average='samples'))
            max_i = Fs.index(max(Fs))
            print('Entire model epoch thresh: %f, max F: %f' %(threshs[max_i], Fs[max_i]))
            
            
            # Check accuracy on the train and val sets every epoch
            # train_acc = check_accuracy(sess, correct_prediction, is_training, train_init_op)
            # val_acc = check_accuracy(sess, correct_prediction, is_training, val_init_op)
            # print('Train accuracy: %f' % train_acc)
            # print('Val accuracy: %f\n' % val_acc)
    

100%|██████████| 40479/40479 [00:00<00:00, 473856.14it/s]

[{'dp': 0.4, 'lr1': 0.001, 'lr2': 1e-05, 'wd': 0.0001}]
listed
{'dp': 0.4, 'lr1': 0.001, 'lr2': 1e-05, 'wd': 0.0001}
(?, 13, 13, 64)
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'



Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


INFO:tensorflow:Restoring parameters from vgg_16.ckpt


INFO:tensorflow:Restoring parameters from vgg_16.ckpt


Starting epoch 1 / 15
Current loss: 8.412778
thresh: 0.050000, max F: 0.479282
Current loss: 8.044341
thresh: 0.500000, max F: 0.575802
Current loss: 8.384157
thresh: 0.500000, max F: 0.571492
Current loss: 8.043993
thresh: 0.500000, max F: 0.619258
Current loss: 8.128809
thresh: 0.500000, max F: 0.600588
Current loss: 8.496967
thresh: 0.500000, max F: 0.617190
Current loss: 9.091754
thresh: 0.500000, max F: 0.640518
Current loss: 8.269986
thresh: 0.500000, max F: 0.610043
Current loss: 8.354798
thresh: 0.500000, max F: 0.610030
Current loss: 8.779675
thresh: 0.500000, max F: 0.573935
Current loss: 7.702882
thresh: 0.500000, max F: 0.586381
Current loss: 8.467604
thresh: 0.500000, max F: 0.651979
Current loss: 8.184071
thresh: 0.500000, max F: 0.614028
Current loss: 8.580548
thresh: 0.500000, max F: 0.637821
Current loss: 8.410450
thresh: 0.500000, max F: 0.605375
Current loss: 8.410397
thresh: 0.500000, max F: 0.602128
Current loss: 8.155311
thresh: 0.500000, max F: 0.579709
Current l

/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


ResourceExhaustedError: OOM when allocating tensor with shape[100,64,112,112]
	 [[Node: gradients_1/vgg_16/pool1/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](vgg_16/conv1/conv1_2/Relu, vgg_16/pool1/MaxPool, gradients_1/vgg_16/conv2/conv2_1/convolution_grad/tuple/control_dependency)]]

Caused by op 'gradients_1/vgg_16/pool1/MaxPool_grad/MaxPoolGrad', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/cs231n/myVE35/lib/python3.5/site-packa

In [ ]:
predicted_logits

In [93]:
actual_labels

[array([[0, 0, 0, ..., 1, 0, 0],
        [0, 1, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 1, 0],
        ..., 
        [1, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0]], dtype=int32), array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        ..., 
        [1, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 1, ..., 0, 1, 0]], dtype=int32), array([[0, 0, 1, ..., 0, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        ..., 
        [1, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0]], dtype=int32), array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        ..., 
        [0, 0, 0, ..., 1, 1, 0],
        [0, 1, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 1, 1, 0]], dtype=int32), array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0

In [84]:
loss_log

[7.7339444,
 7.4191761,
 7.7160192,
 7.9325938,
 8.340826,
 7.9754939,
 7.4352045,
 7.7459483,
 8.023757,
 7.66538]